# Graphs

A graph is a data structure that consists of a set of vertices (or nodes) connected by edges (or links). It is used to represent relationships or connections between pairs of objects. Graphs can be directed (where edges have a direction) or undirected (where edges do not have a direction). They can also be weighted (where edges have values) or unweighted. Graphs are commonly used in various applications, such as social networks, transportation systems, and network topology, to model complex relationships and facilitate analysis.

A graph can be represented in several ways, primarily through:

- **Adjacency List**: A collection of lists or arrays where each list corresponds to a vertex and contains the vertices that are directly connected to it. This representation is more space-efficient for sparse graphs.
- **Adjacency Matrix**: A 2D array where the rows and columns represent vertices, and the cell values indicate whether an edge exists between the vertices. For example, a value of 1 indicates an edge, while 0 indicates no edge. Takes a lot of memory due to being of `O(V^2)` complexity, therefore rarely used.
- **Edge List**: A list of pairs (or tuples) where each pair represents an edge connecting two vertices. This is a simple and straightforward representation.

In [10]:
from typing import List
from queue import Queue

def bfs(
    graph: List[List[int]],
    source: int,
    needle: int) -> List[int] | None:

    q = Queue()
    
    seen = [False] * len(graph)
    prev = [-1] * len(graph)

    seen[source] = True

    q.put(source)

    while not q.empty():
        current = q.get()

        if current == needle:
            break
        
        adjs = graph[current]

        for i in range(len(adjs)):
            if adjs[i] == 0:
                continue

            if seen[i]:
                continue
            
            seen[i] = True
            prev[i] = current
            q.put(i)

    if prev[needle] == -1:
        return

    current = needle

    path = []

    while prev[current] != -1:
        path.append(current)
        current = prev[current]

    return [source] + path[::-1]



#      >(1)<--->(4) ---->(5)
#     /          |       /|
#  (0)     ------|------- |
#     \   v      v        v
#      >(2) --> (3) <----(6)

graph_matrix = [
    [0, 3, 1,  0, 0, 0, 0],
    [0, 0, 0,  0, 1, 0, 0],
    [0, 0, 7,  0, 0, 0, 0],
    [0, 0, 0,  0, 0, 0, 0],
    [0, 1, 0,  5, 0, 2, 0],
    [0, 0, 18, 0, 0, 0, 1],
    [0, 0, 0,  1, 0, 0, 1],
]

assert bfs(graph_matrix, 0, 6) == [
    0,
    1,
    4,
    5,
    6,
]

assert bfs(graph_matrix, 6, 0) is None

In [1]:
class Edge():
    to: int
    weight: int

    def __init__(self, to, weight):
        self.to = to
        self.weight = weight

def walk(
    graph: List[Edge],
    current: int,
    needle: int,
    seen: List[bool],
    path: List[int]
) -> bool:

    if seen[current]:
        return False
    
    seen[current] = True
    
    # pre
    path.append(current)
    if current == needle:
        return True

    # recurse
    edges = graph[current]

    for i in range(len(edges)):
        edge = edges[i]

        if walk(graph, edge.to, needle, seen, path):
            return True

    # post
    path.pop()

    return False


def dfs(
    graph: List[Edge],
    source: int,
    needle: int
) -> List[int] | None:
    seen: List[bool] = [False] * len(graph)
    path: List[int] = []

    walk(graph, source, needle, seen, path)

    if not len(path):
        return
    
    return path

#      >(1)<--->(4) ---->(5)
#     /          |       /|
#  (0)     ------|------- |
#     \   v      v        v
#      >(2) --> (3) <----(6)

adjacency_list = [
    [Edge(1, 3), Edge(2, 1),],
    [Edge(4, 1)],
    [Edge(3, 7)],
    [],
    [Edge(1, 1), Edge(3, 5), Edge(5, 2)],
    [Edge(2, 18), Edge(6, 1)],
    [Edge(3, 1)],
]

assert dfs(adjacency_list, 0, 6) == [
    0,
    1,
    4,
    5,
    6,
]

assert dfs(adjacency_list, 6, 0) is None

NameError: name 'List' is not defined

## Dijkstra's Shortest Path

Dijkstra's Shortest Path is an algorithm used to find the shortest path between nodes in a graph, which may represent, for example, road networks. It works by iteratively selecting the closest unvisited node, updating the distances to its neighbors, and marking it as visited until all nodes have been processed or the shortest path to a specific target node is found. The algorithm is efficient for graphs with non-negative edge weights and is widely used in various applications, such as GPS navigation and network routing.

In [8]:
from typing import List
from queue import PriorityQueue

def has_unvisited(seen: List[bool], dists: List[float]) -> bool:
    for idx, s in enumerate(seen):
        if not s and dists[idx] < float("inf"):
            return True
    return False

def get_lowest_unvisited(seen: List[bool], dists: List[float]) -> int:
    idx = -1
    lowest_distance = float("inf")

    for i, s in enumerate(seen):
        if s:
            continue

        if lowest_distance > dists[i]:
            lowest_distance = dists[i]
            idx = i

    return idx

# O(V^2+E) using arrays
def dijkstra_list_slower(source: int, sink: int, adjs_list: List[Edge]):
    seen = [False] * len(adjs_list)
    prev = [-1] * len(adjs_list)

    dists = [float("inf")] * len(adjs_list)
    dists[source] = 0

    while has_unvisited(seen, dists):
        current = get_lowest_unvisited(seen, dists)
        seen[current] = True

        adjs = adjs_list[current]

        for i in range(len(adjs)):
            edge = adjs[i]
            if seen[edge.to]:
                continue

            dist = dists[current] + edge.weight
            if dist < dists[edge.to]:
                dists[edge.to] = dist
                prev[edge.to] = current

    out = []
    current = sink

    while prev[current] != -1:
        out.append(current)
        current = prev[current]

    out.append(source)
    return out[::-1]

# O(logV(V+E)) using PriorityQueue
def dijkstra_list_faster(source: int, sink: int, adjs_list: List[Edge]):
    seen = [False] * len(adjs_list)
    prev = [-1] * len(adjs_list)

    dists = [float("inf")] * len(adjs_list)
    dists[source] = 0

    q = PriorityQueue()
    q.put(source)

    while not q.empty():
        current = q.get()
        seen[current] = True

        adjs = adjs_list[current]

        for i in range(len(adjs)):
            edge = adjs[i]
            if seen[edge.to]:
                continue

            q.put(edge.to)

            dist = dists[current] + edge.weight
            if dist < dists[edge.to]:
                dists[edge.to] = dist
                prev[edge.to] = current

    out = []
    current = sink

    while prev[current] != -1:
        out.append(current)
        current = prev[current]

    out.append(source)
    return out[::-1]


#       (1) --- (4) ---- (5)
#     /  |       |       /|
#  (0)   | ------|------- |
#     \  |/      |        |
#       (2) --- (3) ---- (6)

adjacency_list_2 = [
    [Edge(1,3), Edge(2,1)],
    [Edge(0,3), Edge(2,4), Edge(4,1)],
    [Edge(1,4), Edge(3,7), Edge(0,1)],
    [Edge(2,7), Edge(4,5), Edge(6,1)],
    [Edge(1,1), Edge(3,5), Edge(5,2)],
    [Edge(6,1), Edge(4,2), Edge(2,18)],
    [Edge(3,1), Edge(5,1)],
]

assert dijkstra_list_slower(0, 6, adjacency_list_2) == [0, 1, 4, 5, 6]
assert dijkstra_list_faster(0, 6, adjacency_list_2) == [0, 1, 4, 5, 6]